In [1]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import cross_validation
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pandas as pd
import numpy as np
import xgboost as xg
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Normalizer
from tpot.builtins import StackingEstimator

%matplotlib inline

/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dummy_df = pd.read_csv("../results/dummied_c_elegans_30mers.csv")
print(dummy_df.columns.tolist()[-1])
X, y = dummy_df.iloc[:,:-1].values, dummy_df.iloc[:,-1].values

target


In [7]:
model = GradientBoostingRegressor()
base_cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
(sum(abs(base_cross_val))/len(base_cross_val))

/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_er

0.031096936539194216

In [8]:
model = xg.XGBRegressor(n_estimators=1000)
xg1000_cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
(sum(abs(xg1000_cross_val))/len(xg1000_cross_val))

/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_er

0.03236412042123666

In [6]:
model = make_pipeline(
    Normalizer(norm="max"),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=False, max_features=0.95, min_samples_leaf=4, min_samples_split=9, n_estimators=100)),
    RandomForestRegressor(bootstrap=True, max_features=0.35, min_samples_leaf=1, min_samples_split=8, n_estimators=100)
)
pipeline_cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
(sum(abs(pipeline_cross_val))/len(pipeline_cross_val))

/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/deadhand/anaconda3/envs/hackseq_2017/lib/python2.7/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_er

0.029705100239749833

In [71]:
[base_]

2.3380535190836502

In [44]:
[i for i,j in enumerate(dummy_df.columns.values) if j == "lfc"]

[149]

In [74]:
! git add baseline.ipynb

remote: Counting objects: 94, done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 94 (delta 21), reused 89 (delta 16), pack-reused 0
Unpacking objects: 100% (94/94), done.
From github.com:hackseq/2017_project_8
   61019cb..04eb77e  master     -> origin/master
Updating 61019cb..04eb77e
Fast-forward
 ...CRISPR_sgRNA_Database_Responses_Anon_30mers.csv |   74 +
 .../analysis/hackseq_featurization.py              |    2 +-
 ...d_+_UTR_+_2000_upstream_+_2000_downstream.fasta |   57 +
 ...d_+_UTR_+_2000_upstream_+_2000_downstream.fasta |   57 +
 ...d_+_UTR_+_2000_upstream_+_2000_downstream.fasta |   63 +
 ...d_+_UTR_+_2000_upstream_+_2000_downstream.fasta |   92 +
 ...d_+_UTR_+_1072_upstream_+_2000_downstream.fasta |   56 +
 ...d_+_UTR_+_2000_upstream_+_2000_downstream.fasta |   89 +
 ...iced_+_UTR_+_939_upstream_+_78_downstream.fasta |   34 +
 ...ed_+_UTR_+_897_upstream_+_2000_downstream.fasta |   62 +
 .../C32E8.11unspliced_+_UTR_+_2000_upstream.fasta  |  143 +
 ...d_+_UTR_